In [ ]:
import marlin as mar

In [ ]:
handler_1 = mar.handlerCore()

In [ ]:
handler_1.connect()

In [ ]:
handler_1.set_valve_state("PFA(half-MeAc)", 0)

In [ ]:
handler_1.set_pump_state(0)

In [ ]:
Ti6 = scopeCore(
    "./DE_config.cfg",
    "./logfile.txt",
    camera_name="Iris15",
    shutter_name="LightEngine",
    xystage_name="XYStage",
    focus_name="ZDrive",
)

In [ ]:
Ti6.mmc.setConfig("FISH_Channels", "BF")

In [ ]:
Ti6.liveview()

In [ ]:
test_grid = Ti5.set_grid(44, 3)

In [ ]:
len(test_grid)

In [ ]:
Ti5.mmc.setXYPosition(test_grid[0][0], test_grid[0][1])

In [ ]:
for t in range(10):
    Ti5.multipoint_aq(
        test_grid,
        ["BF", "RFP", "Cy5", "Cy7"],
        t,
        output_folder="C:/Users/System 4/Documents/test_output/",
    )

In [ ]:
def liveview(self, img_size=(12, 12), low=None, high=None):  # W,interval=0.5):
    while True:
        try:
            while self.mmc.deviceBusy(self.camera_name):
                time.sleep(0.005)

            im1 = self.snap_image()
            clear_output(wait=True)
            plt.figure(figsize=img_size)
            if low == None or high == None:
                plt.imshow(im1, interpolation="None", cmap="gray")
            else:
                plt.imshow(im1, interpolation="None", cmap="gray", vmin=low, vmax=high)
            plt.show()
        except KeyboardInterrupt:
            break
    while self.mmc.deviceBusy(self.camera_name):
        time.sleep(0.01)

In [ ]:
from time import sleep
import h5py

class FISH_scheduler:
    def __init__(self,handlerInstance,scopeInstance,fast_speed=2000,medium_speed=300,slow_speed=100,\
                 mins_fast_speed=4.,mins_medium_speed=5.,channels=["BF","RFP","Cy5","Cy7"],\
                output_folder="./"):
        self.handlerInstance = handlerInstance
        self.scopeInstance = scopeInstance
        self.fast_speed = fast_speed
        self.medium_speed = medium_speed
        self.slow_speed = slow_speed
        
        self.secs_fast_speed = int(mins_fast_speed*60)
        self.secs_medium_speed = int(mins_medium_speed*60)
        
        self.channels = channels
        self.output_folder = output_folder
        
    def wait_for(self,num_secs):
        for t in range(num_secs):
            sleep(1.)
            
    def load_reagent(self,reagent_name):
        print reagent_name
        
        self.handlerInstance.set_pump_state(0)
        self.handlerInstance.set_valve_state(reagent_name,1)
        self.handlerInstance.set_pump_state(self.fast_speed)
        
        self.wait_for(self.secs_fast_speed)
        
        self.handlerInstance.set_pump_state(self.medium_speed)
        self.handlerInstance.set_valve_state(reagent_name,0)
        
        self.wait_for(self.secs_medium_speed)
        
        return True
        
    def init_fixation(self):
        self.load_reagent("PFA(half-MeAc)")
        print "Initialized."
        
    def continue_fixation(self):
        self.load_reagent("EtOH(MeAc)")
        self.handlerInstance.set_pump_state(self.slow_speed)
        self.wait_for(45*60)
        self.load_reagent("PFA(half-MeAc)")
        print "Fixed."
        
    def perform_cycle(self,cycle_num,no_cleave=False):
        reagent_name = "Probe " + str(cycle_num)
        
        if not no_cleave:
            self.load_reagent("Cleave")
            self.wait_for(10*60)
            
        self.load_reagent("SSC")
        self.load_reagent(reagent_name)
        self.wait_for(15*60)
        self.load_reagent("Image")
        self.wait_for(5*60)
        self.handlerInstance.set_pump_state(self.slow_speed)        
        
    def run(self,grid_coords,num_cycles=8):
        first_x,first_y = grid_coords[0]
        self.scopeInstance.mmc.setXYPosition(first_x,first_y)
        
        img = self.scopeInstance.snap_image()
        
        with h5py.File(self.output_folder + "initial.hdf5","w") as h5pyfile:
            hdf5_dataset = h5pyfile.create_dataset("data", data=img, chunks=(128,128), dtype='uint16')
        
        self.init_fixation()
        
        img = self.scopeInstance.snap_image()
        
        with h5py.File(self.output_folder + "init_fixation.hdf5","w") as h5pyfile:
            hdf5_dataset = h5pyfile.create_dataset("data", data=img, chunks=(128,128), dtype='uint16')
        
        self.continue_fixation()
        
        img = self.scopeInstance.snap_image()
        
        with h5py.File(self.output_folder + "fixed.hdf5","w") as h5pyfile:
            hdf5_dataset = h5pyfile.create_dataset("data", data=img, chunks=(128,128), dtype='uint16')
        
        for c in range(1,num_cycles+1):
            if c == 1:
                self.perform_cycle(c,no_cleave=True)
            else:
                self.perform_cycle(c)
            
            print "Imageing..."
            
            self.scopeInstance.multipoint_aq(grid_coords,self.channels,c,output_folder=self.output_folder)

In [ ]:
E:\Daniel\FISH_barcoding_project\2020-01-31_Auto_FISH_test

In [ ]:
fish = FISH_scheduler(
    handler_1,
    Ti5,
    fast_speed=2000,
    medium_speed=300,
    slow_speed=100,
    mins_fast_speed=4.0,
    mins_medium_speed=5.0,
    channels=["BF", "RFP", "Cy5", "Cy7"],
    output_folder="E:/Daniel/FISH_barcoding_project/2020-01-31_Auto_FISH_test/",
)

In [ ]:
fish.run(test_grid, num_cycles=8)

In [ ]:
from pylab import *

ion()

import numpy as np
import matplotlib
import MMCorePy
from IPython.display import clear_output
import matplotlib.pyplot as plt
import time

%matplotlib inline

### Init Nikon
 - make sure the scope is not escaped

In [ ]:
mmc = MMCorePy.CMMCore()

In [ ]:
mmc.loadSystemConfiguration("./master_config.cfg")

In [ ]:
mmc.setPrimaryLogFile("./mmcpy_log/log.txt")

### Snap and plot an image

In [ ]:
mmc.getLoadedDevices()

In [ ]:
mmc.getDevicePropertyNames("Turret1Shutter")

In [ ]:
mmc.setProperty("Turret1Shutter", "State", True)

In [ ]:
mmc.setShutterDevice("LightEngine")

### Live View

In [ ]:
def liveview(camera, img_size=(12, 12), low=None, high=None):  # W,interval=0.5):
    mmc.setCameraDevice(camera)
    while True:
        try:
            while mmc.deviceBusy(camera):
                time.sleep(0.005)
            mmc.snapImage()
            im1 = mmc.getImage()
            clear_output(wait=True)
            plt.figure(figsize=img_size)
            if low == None or high == None:
                plt.imshow(im1, interpolation="None", cmap="gray")
            else:
                plt.imshow(im1, interpolation="None", cmap="gray", vmin=low, vmax=high)
            plt.show()
        except KeyboardInterrupt:
            break
    while mmc.deviceBusy(camera):
        time.sleep(0.01)


#         slp = interval - (time.time() - ti)
#         time.sleep(slp)

In [ ]:
mmc.setShutterOpen("LightEngine", False)

In [ ]:
liveview("Iris15", low=None, high=None)

### Snap and Contrast

In [ ]:
def snap_image(camera, img_size=(12, 12)):
    mmc.setCameraDevice(camera)
    mmc.snapImage()
    im1 = mmc.getImage()
    return im1


def auto_contrast(img, low_percentile=0, high_percentile=100):
    low = np.percentile(img, low_percentile)
    high = np.percentile(img, high_percentile)
    return low, high


def plot_img(img, low, high, img_size=(12, 12)):
    clear_output(wait=True)
    plt.figure(figsize=img_size)
    plt.imshow(im1, interpolation="None", vmin=low, vmax=high)
    plt.show()

In [ ]:
im1 = snap_image("Iris15")
low, high = auto_contrast(im1)
plot_img(im1, low, high)

### Imaging Parameters

In [ ]:
mmc.getDevicePropertyNames("Iris15")

In [ ]:
mmc.getProperty("Iris15", "Exposure")

#### Exposure

In [ ]:
mmc.setExposure("Iris15", 200.0)

In [ ]:
mmc.getExposure("Iris15")

#### Binning

In [ ]:
mmc.getAllowedPropertyValues("Iris15", "Binning")

In [ ]:
mmc.setProperty("Iris15", "Binning", "1x1")

In [ ]:
mmc.getProperty("Iris15", "Binning")

#### metadata?

In [ ]:
mmc.getAllowedPropertyValues("Iris15", "MetadataEnabled")

In [ ]:
mmc.getProperty("Iris15", "MetadataEnabled")

In [ ]:
mmc.snapImage()

In [ ]:
test = mmc.getImage()

### Shutter

#### Changing SpectraIII Shutter

In [ ]:
mmc.setShutterOpen("LightEngine", False)

#### Changing Epi1 Shutter

In [ ]:
mmc.setShutterOpen("Turret1Shutter", True)

### Turret

In [ ]:
mmc.getDevicePropertyNames("FilterTurret1")

In [ ]:
mmc.getAllowedPropertyValues("FilterTurret1", "Label")

In [ ]:
mmc.getAllowedPropertyValues("FilterTurret1", "State")

In [ ]:
mmc.setProperty("FilterTurret1", "State", "3")

In [ ]:
mmc.getProperty("FilterTurret1", "Label")

In [ ]:
mmc.getProperty("FilterTurret1", "State")

### Emission Wheel

In [ ]:
def set_filter_wheel(mmc, state_idx):
    filter_wheel_states = [
        "FW_Empty",
        "FW_FITC",
        "FW_TRITC",
        "FW_Cy5",
        "FW_CFPHQ",
        "FW_YFPHQ",
    ]
    all_states = set(range(6))
    off_states = list(all_states - set([state_idx]))
    for i in off_states:
        mmc.setProperty(filter_wheel_states[i], "Volts", 0.0)
    mmc.setProperty(filter_wheel_states[state_idx], "Volts", 5.0)

In [ ]:
set_filter_wheel(mmc, 4)

### Spectra III

In [ ]:
mmc.getDevicePropertyNames("SpectraIII")

In [ ]:
channels = ["VIOLET", "BLUE", "CYAN", "TEAL", "GREEN", "YELLOW", "RED", "NIR"]
for channel in channels:
    channel_int = channel + "_Intensity"
    print(mmc.getProperty("SpectraIII", channel_int))

In [ ]:
mmc.setProperty("SpectraIII", "RED", "0")

In [ ]:
mmc.getProperty("SpectraIII", "RED")

In [ ]:
mmc.setProperty("SpectraIII", "RED_Intensity", "0")

In [ ]:
mmc.getProperty("SpectraIII", "RED_Intensity")

### Nosepiece

In [ ]:
mmc.getDevicePropertyNames("Nosepiece")

In [ ]:
mmc.getAllowedPropertyValues("Nosepiece", "Label")

In [ ]:
mmc.getAllowedPropertyValues("Nosepiece", "State")

In [ ]:
mmc.setProperty("Nosepiece", "State", "2")

In [ ]:
mmc.getProperty("Nosepiece", "Label")

### XYStage

In [ ]:
mmc.getXYStageDevice()

In [ ]:
mmc.getXYPosition()

In [ ]:
mmc.deviceBusy("XYStage")

#### Example grid loop

In [ ]:
x_ori, y_ori = mmc.getXYPosition()
for x in range(0, 2000, 500):
    for y in range(0, 2000, 500):
        while mmc.deviceBusy("XYStage"):
            time.sleep(0.1)
            pass
        mmc.setXYPosition(x_ori + x, y_ori + y)

### ZStage

In [ ]:
mmc.getPosition()

In [ ]:
mmc.getFocusDirection("ZDrive")

In [ ]:
mmc.setRelativePosition("ZDrive", -5000.0)

In [ ]:
mmc.getFocusDirection("ZDrive")

In [ ]:
mmc.getFocusDevice()

### PFS

In [ ]:
mmc.getDevicePropertyNames("PFS")

In [ ]:
mmc.getAllowedPropertyValues("PFS", "FocusMaintenance")

In [ ]:
mmc.setProperty("PFS", "FocusMaintenance", "Off")

In [ ]:
mmc.getProperty("PFS", "FocusMaintenance")

In [ ]:
mmc.deviceBusy("PFS")

### PFS Offset

In [ ]:
mmc.getFocusDirection("PFSOffset")

In [ ]:
mmc.setRelativePosition("PFSOffset", 1.0)

### Cool LED

In [ ]:
mmc.getDevicePropertyNames("CoolLED")

In [ ]:
mmc.setProperty("CoolLED", "Volts", "0")

In [ ]:
mmc.getProperty("CoolLED", "Volts")

### Testing Multichannel

In [ ]:
mmc.setConfig("FISH_channels", "RFP")

In [ ]:
mmc.setConfig("FISH_channels", "Cy5")

In [ ]:
mmc.setConfig("FISH_channels", "Cy7")

In [ ]:
mmc.setShutterOpen("SpectraIII", False)

In [ ]:
def set_grid(num_col, num_row, col_step=333.0, row_step=686.0):
    grid_coords = []

    x_ori, y_ori = mmc.getXYPosition()
    start_left = True

    for row in range(num_row):
        y_disp = row * row_step

        if start_left:
            for col in range(num_col):
                x_disp = col * (-col_step)
                current_coord = (x_ori + x_disp, y_ori + y_disp)
                grid_coords.append(current_coord)
            start_left = False
        else:
            for col in range(num_col):
                x_disp = (num_col - col - 1) * (-col_step)
                current_coord = (x_ori + x_disp, y_ori + y_disp)
                grid_coords.append(current_coord)
            start_left = True

    return grid_coords

In [ ]:
test_grid = set_grid(10, 3)

In [ ]:
test_grid[0]

In [ ]:
mmc.setXYPosition(test_grid[0][0], test_grid[0][1])

In [ ]:
import numpy as np

channels = ["BF", "RFP", "Cy5", "Cy7"]
all_imgs = []

for x_coord, y_coord in test_grid:
    while mmc.deviceBusy("XYStage"):
        time.sleep(0.1)
        pass
    mmc.setXYPosition(x_coord, y_coord)

    imgs = []
    for channel in channels:
        while mmc.systemBusy():
            time.sleep(0.1)
            pass
        mmc.setConfig("FISH_channels", channel)
        while mmc.systemBusy():
            time.sleep(0.1)
            pass
        mmc.setExposure("BSI Prime", 30.0)
        mmc.setShutterOpen("SpectraIII", True)
        mmc.snapImage()
        mmc.setShutterOpen("SpectraIII", False)
        img = mmc.getImage()
        imgs.append(img)

    all_imgs.append(imgs)
all_imgs = np.array(all_imgs)

In [ ]:
all_imgs[0]

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(all_imgs[,0])

In [ ]:
import serial
import serial.tools.list_ports
import time
from time import sleep

In [ ]:
class arduino:
    def __init__(self,handshakes=2):
        self.valvestate = 0
        self.pumpstate = 0
        self.titanxstates = [0 for i in range(5)]
        self.handshakes = handshakes
        self.titanx_states = {"Probe 1" : [0,2,1,12,0],"Probe 2": [0,3,1,12,0],"Probe 3": [0,4,1,12,0],\
                        "Probe 4": [0,5,1,12,0],"Probe 5": [0,6,1,12,0],"Probe 6" : [0,7,1,12,0],\
                        "Probe 7" : [0,8,1,12,0],"Probe 8" : [0,9,1,12,0],"Probe 9" : [0,10,1,12,0],\
                        "Probe 10" : [0,11,1,12,0],"Probe 11" : [0,12,1,12,0],"Probe 12" : [0,1,2,12,0],\
                        "Probe 13" : [0,1,3,12,0],"Probe 14" : [0,1,4,12,0],"Probe 15" : [0,1,5,12,0],\
                       "Probe 16" : [0,1,6,12,0],"Probe 17" : [0,1,7,12,0],"Probe 18" : [0,1,8,12,0],\
                        "Probe 19" : [0,1,9,12,0],"Probe 20" : [0,1,10,12,0],"Probe 21" : [0,1,11,12,0],\
                        "Probe 22" : [0,1,12,12,0],"Probe 23" : [0,1,1,1,0],"Probe 24" : [0,1,1,2,0],\
                        "SSC" : [0,1,1,8,0],"PFA" : [0,1,1,9,0],"EtOH" : [0,1,1,10,0],"Image" : [0,1,1,11,0],\
                        "Cleave" : [0,1,1,12,0]}
        
    def get_heartbeat(self,comport,connect_code="MARLIN",timeout=10.):
        try:
            ti = time.time()
            no_timeout = True
            s = serial.Serial(comport,9600,timeout=0.5)
            readcmd = "5\n".encode('ascii')
            while no_timeout:
                s.write(readcmd)
                returnedstr = s.read_until()
                t_elapsed = time.time() - ti                    
                if len(returnedstr) > 0:
                    no_timeout = False
                elif t_elapsed>timeout:
                    no_timeout = False
            s.close()
            if returnedstr == "MARLIN":
                return True
            else:
                return False
        except (OSError, serial.SerialException):
            return False 
        
    def connect(self,connect_code="MARLIN",timeout=10.):
        ports = ['COM%s' % (i + 1) for i in range(256)]
        result = []
        for port in ports:
            heartbeat = self.get_heartbeat(port,connect_code=connect_code,timeout=timeout)
            if heartbeat:
                result.append(port)
        if len(result) == 0:
            raise ValueError("No MARLIN detected.")
        elif len(result) == 1:
            ti = time.time()
            no_timeout = True
            self.serial_handle = serial.Serial(result[0], 9600, timeout=0.5)
            readcmd = "5\n".encode('ascii')
            while no_timeout:
                self.serial_handle.write(readcmd)
                returnedstr = self.serial_handle.read_until()
                t_elapsed = time.time() - ti
                if len(returnedstr) > 0:
                    no_timeout = False
                elif t_elapsed>timeout:
                    no_timeout = False
            if returnedstr == "MARLIN":
                self.serial_handle.timeout = 10.
                print("Connected.")
            else:
                raise ValueError("MARLIN connection timeout.")
        else:
            raise ValueError("More than one MARLIN detected.")
        
        
    def updatestate(self,valvestate,pumpstate,titanxstates):
        self.valvestate = valvestate
        self.pumpstate = pumpstate
        self.titanxstates = titanxstates
        
    def sendstate(self,valvestate,pumpstate,titanxstates):## Note need to rewrite arduino code for handshake using
    # newlines
        self.updatestate(valvestate,pumpstate,titanxstates)
        no_handshake = True
        handshake_failed = False
        handshake_attempts = 0

        while no_handshake:
            valvestr = "4" + str(self.valvestate)
            pumpstr = str(self.pumpstate)
            pumpstr = "3" + ("0"*(4-len(pumpstr)) + pumpstr)

            titanxstrlist = []

            for titannum,titanxstate in enumerate(self.titanxstates):
                if titanxstate != 0:
                    titanxstr = str(titanxstate)
                    titanxstr = "2" + str(titannum) + ("0"*(2-len(titanxstr)) + titanxstr)
                    titanxstrlist.append(titanxstr)

            cmdlist = [""] + [valvestr] + [pumpstr] + titanxstrlist

            for cmd in cmdlist:
                sendstr = cmd + '\n'
                statestr = sendstr.encode('ascii')
                self.serial_handle.write(statestr)
                time.sleep(0.25)

            readcmd = "0\n".encode('ascii')
            self.serial_handle.write(readcmd)
            returnedstr = self.serial_handle.read_until()[:-1]
            self.serial_handle.reset_output_buffer()
            self.serial_handle.reset_input_buffer()
            checkstr = "[" + ",".join([str(state) for state in self.titanxstates]) + "];" + str(self.valvestate) + ";" + str(self.pumpstate)
            
            ###OVERRIDE FOR TESTING###
            
#             no_handshake = False
#             print checkstr.strip()
            print returnedstr.strip()
            
            if returnedstr.strip() == checkstr.strip():
                no_handshake = False
            handshake_attempts += 1
            if handshake_attempts >= self.handshakes:
                raise Exception("Handshake failed.")

    def set_valve_state(self,titanx_state_name,valvestate):
        titanxstates = self.titanx_states[titanx_state_name]
        self.sendstate(valvestate,self.pumpstate,titanxstates)
    
    def set_pump_state(self,pumpstate):
        self.sendstate(self.valvestate,pumpstate,self.titanxstates)
        
        

In [ ]:
ard = arduino()

In [ ]:
ard.connect()

In [ ]:
ard.set_valve_state("PFA", 1)

In [ ]:
ard.set_pump_state(0)

In [ ]:
# cleaning
for state_name, _ in handler_1.titanx_states.items():
    handler_1.set_pump_state(0)
    handler_1.set_valve_state(state_name, 0)
    handler_1.set_pump_state(2000)
    sleep(90.0)
handle.set_pump_state(0)
handler_1.set_valve_state("Image", 1)
handler_1.set_pump_state(2000)
sleep(5.0 * 60.0)
handler_1.set_pump_state(0)
handler_1.set_valve_state("Image", 0)

In [ ]:
import numpy as np

channels = ["BF", "RFP", "Cy5", "Cy7"]
all_imgs = []

for x_coord, y_coord in test_grid:
    while mmc.deviceBusy("XYStage"):
        time.sleep(0.1)
        pass
    mmc.setXYPosition(x_coord, y_coord)

    imgs = []
    for channel in channels:
        while mmc.systemBusy():
            time.sleep(0.1)
            pass
        mmc.setConfig("FISH_channels", channel)
        while mmc.systemBusy():
            time.sleep(0.1)
            pass
        mmc.setExposure("BSI Prime", 30.0)
        mmc.setShutterOpen("SpectraIII", True)
        mmc.snapImage()
        mmc.setShutterOpen("SpectraIII", False)
        img = mmc.getImage()
        imgs.append(img)

    all_imgs.append(imgs)
all_imgs = np.array(all_imgs)

In [ ]:
import pandas as pd

In [ ]:
import xml.etree.ElementTree as ET
import xmltodict
import json

tree = ET.parse("multipoints_this_is_just_a_test.xml")
xml_data = tree.getroot()

xmlstr = ET.tostring(xml_data, encoding="utf8", method="xml")


data_dict = dict(xmltodict.parse(xmlstr))

with open("new_data_2.json", "w+") as json_file:
    json.dump(data_dict, json_file, indent=4, sort_keys=True)

In [ ]:
scopeCore()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import MMCorePy
from IPython.display import clear_output
import matplotlib.pyplot as plt
import time

import h5py


class scopeCore:
    def __init__(
        self,
        configpath,
        logpath,
        camera_name="BSI Prime",
        shutter_name="SpectraIII",
        xystage_name="XYStage",
        focus_name="ZDrive",
    ):
        self.mmc = MMCorePy.CMMCore()
        self.mmc.loadSystemConfiguration(configpath)
        self.mmc.setPrimaryLogFile(logpath)
        self.mmc.setCameraDevice(camera_name)

        self.camera_name = camera_name
        self.shutter_name = shutter_name
        self.xystage_name = xystage_name
        self.focus_name = focus_name

    def snap_image(self, img_size=(12, 12)):
        self.mmc.snapImage()
        im1 = self.mmc.getImage()
        return im1

    def auto_contrast(self, img, low_percentile=0, high_percentile=100):
        low = np.percentile(img, low_percentile)
        high = np.percentile(img, high_percentile)
        return low, high

    def plot_img(self, img, low, high, img_size=(12, 12)):
        clear_output(wait=True)
        plt.figure(figsize=img_size)
        plt.imshow(im1, interpolation="None", vmin=low, vmax=high)
        plt.show()

    def liveview(self, img_size=(12, 12), low=None, high=None):  # W,interval=0.5):
        while True:
            try:
                while self.mmc.deviceBusy(self.camera_name):
                    time.sleep(0.005)

                im1 = self.snap_image()
                clear_output(wait=True)
                plt.figure(figsize=img_size)
                if low == None or high == None:
                    plt.imshow(im1, interpolation="None", cmap="gray")
                else:
                    plt.imshow(
                        im1, interpolation="None", cmap="gray", vmin=low, vmax=high
                    )
                plt.show()
            except KeyboardInterrupt:
                break
        while self.mmc.deviceBusy(self.camera_name):
            time.sleep(0.01)

    def set_grid(self, num_col, num_row, col_step=333.0, row_step=686.0):
        grid_coords = []

        x_ori, y_ori = self.mmc.getXYPosition()
        start_left = True

        for row in range(num_row):
            y_disp = row * row_step

            if start_left:
                for col in range(num_col):
                    x_disp = col * (-col_step)
                    current_coord = (x_ori + x_disp, y_ori + y_disp)
                    grid_coords.append(current_coord)
                start_left = False
            else:
                for col in range(num_col):
                    x_disp = (num_col - col - 1) * (-col_step)
                    current_coord = (x_ori + x_disp, y_ori + y_disp)
                    grid_coords.append(current_coord)
                start_left = True

        return grid_coords

    def multipoint_aq(
        self,
        grid_coords,
        config_list,
        timepoint,
        output_folder="./",
        group_name="FISH_channels",
    ):

        ### Make sure configs are valid ###
        undefined_configs = []
        for config in config_list:
            config_defined = self.mmc.isConfigDefined(group_name, config)
            if not config_defined:
                undefined_configs.append(config)

        if len(undefined_configs) > 0:
            raise ValueError(
                "The following configs are undefined: " + ", ".join(undefined_configs)
            )

        ### Gather basic metadata ###

        t_start = time.time()
        x_dim = self.mmc.getProperty(self.camera_name, "X-dimension")
        y_dim = self.mmc.getProperty(self.camera_name, "Y-dimension")

        ## Note change the write to disk later ##

        imgs = []
        imgs_metadata = []

        x_coord, y_coord = grid_coords[0]
        self.mmc.setXYPosition(x_coord, y_coord)

        for fov_num, (x_coord, y_coord) in enumerate(grid_coords):
            while self.mmc.deviceBusy(self.xystage_name):
                time.sleep(0.1)
                pass
            self.mmc.setXYPosition(x_coord, y_coord)

            for config in config_list:
                while self.mmc.systemBusy():
                    time.sleep(0.1)
                    pass
                self.mmc.setConfig(group_name, config)

                #                 ### put write here because it is likely the slow step ###

                #                 for img_num in range(len(imgs)):
                #                     img = imgs[img_num]
                #                     metadata_entry = imgs_metadata[img_num]

                #                     with h5py.File(output_folder + "fov=" + str(metadata_entry["fov"]) + "_config=" + str(metadata_entry["config"]) + "_t=" + str(timepoint),"w") as h5pyfile:
                #                         hdf5_dataset = h5pyfile.create_dataset("data", data=img, chunks=(128,128), dtype='uint16')
                #                         all_metadata.append(metadata_entry)

                while self.mmc.systemBusy():
                    time.sleep(0.1)
                    pass
                self.mmc.setShutterOpen(self.shutter_name, True)
                self.mmc.snapImage()
                self.mmc.setShutterOpen(self.shutter_name, False)

                read_x_coord, read_y_coord = self.mmc.getXYPosition(self.xystage_name)
                read_z_coord = self.mmc.getPosition(self.focus_name)
                current_time = time.time() - t_start

                metadata_entry = {
                    "fov": fov_num,
                    "config": config,
                    "x": read_x_coord,
                    "y": read_y_coord,
                    "z": read_z_coord,
                    "t": current_time,
                }
                img = self.mmc.getImage()

                imgs.append(img)
                imgs_metadata.append(metadata_entry)

        x_coord, y_coord = grid_coords[0]
        self.mmc.setXYPosition(x_coord, y_coord)

        for img_num in range(len(imgs)):
            img = imgs[img_num]
            metadata_entry = imgs_metadata[img_num]

            with h5py.File(
                output_folder
                + "fov="
                + str(metadata_entry["fov"])
                + "_config="
                + str(metadata_entry["config"])
                + "_t="
                + str(timepoint),
                "w",
            ) as h5pyfile:
                hdf5_dataset = h5pyfile.create_dataset(
                    "data", data=img, chunks=(128, 128), dtype="uint16"
                )

        metadata = pd.DataFrame.from_dict(imgs_metadata)
        metadata.to_hdf(
            output_folder + "metadata_" + str(timepoint) + ".hdf5", key="data", mode="w"
        )